In [ ]:
import Pkg
Pkg.activate(".")

In [1]:
using Distributed

In [2]:
addprocs(1)

1-element Vector{Int64}:
 2

In [3]:
@everywhere using Serialization

In [4]:
fn(x) = x+1

fn (generic function with 1 method)

In [5]:
workers()

1-element Vector{Int64}:
 2

In [6]:
x = remotecall(fn, 2, 1)

Future(2, 1, 5, ReentrantLock(nothing, 0x00000000, 0x00, Base.GenericCondition{Base.Threads.SpinLock}(Base.IntrusiveLinkedList{Task}(nothing, nothing), Base.Threads.SpinLock(0)), (0, 139838623066112, 2)), nothing)

In [7]:
fetch(x)

LoadError: On worker 2:
UndefVarError: `#fn` not defined
Stacktrace:
  [1] [0m[1mdeserialize_datatype[22m
[90m    @[39m [90m~/local/juliaup/juliaup/julia-1.10.0+0.x64.linux.gnu/share/julia/stdlib/v1.10/Serialization/src/[39m[90m[4mSerialization.jl:1399[24m[39m
  [2] [0m[1mhandle_deserialize[22m
[90m    @[39m [90m~/local/juliaup/juliaup/julia-1.10.0+0.x64.linux.gnu/share/julia/stdlib/v1.10/Serialization/src/[39m[90m[4mSerialization.jl:867[24m[39m
  [3] [0m[1mdeserialize[22m
[90m    @[39m [90m~/local/juliaup/juliaup/julia-1.10.0+0.x64.linux.gnu/share/julia/stdlib/v1.10/Serialization/src/[39m[90m[4mSerialization.jl:814[24m[39m
  [4] [0m[1mhandle_deserialize[22m
[90m    @[39m [90m~/local/juliaup/juliaup/julia-1.10.0+0.x64.linux.gnu/share/julia/stdlib/v1.10/Serialization/src/[39m[90m[4mSerialization.jl:874[24m[39m
  [5] [0m[1mdeserialize[22m
[90m    @[39m [90m~/local/juliaup/juliaup/julia-1.10.0+0.x64.linux.gnu/share/julia/stdlib/v1.10/Seria

In [8]:
@everywhere fn(x) = x+1

In [9]:
x = remotecall(fn, 2, 1)

Future(2, 1, 9, ReentrantLock(nothing, 0x00000000, 0x00, Base.GenericCondition{Base.Threads.SpinLock}(Base.IntrusiveLinkedList{Task}(nothing, nothing), Base.Threads.SpinLock(0)), (8, 139838686509568, 139838686509648)), nothing)

In [10]:
fetch(x)

2

In [11]:
y(x) = x+2

y (generic function with 1 method)

In [12]:
function sendfunc(f::Function, dest::Int64, mod::Union{Module, Nothing}=nothing)
    if isnothing(mod)
        mod = parentmodule(f)
    end
    # get fully-qualified name of function
    fname = Symbol(f)
    mname = Symbol(mod)
    # sender serializes function
    buf = IOBuffer()
    Serialization.serialize(buf, methods(eval(:($mname.$fname))))
    # receiver deserializes function
    Distributed.remotecall_eval(
        mod, [dest], quote
            function $fname end 
            Serialization.deserialize(seekstart($buf))
        end
    )
end

sendfunc (generic function with 2 methods)

In [13]:
sendfunc(y, 2)

In [14]:
x = remotecall(y, 2, 2)

Future(2, 1, 13, ReentrantLock(nothing, 0x00000000, 0x00, Base.GenericCondition{Base.Threads.SpinLock}(Base.IntrusiveLinkedList{Task}(nothing, nothing), Base.Threads.SpinLock(0)), (8, 139838887007888, 139838864451776)), nothing)

In [15]:
fetch(x)

4

In [16]:
parentmodule(fn)

Main

In [17]:
function z(x)
    sleep(x)
    return x
end

z (generic function with 1 method)

In [18]:
sendfunc(z, 2)

In [28]:
x = remotecall(z, 2, 20)

Future(2, 1, 23, ReentrantLock(nothing, 0x00000000, 0x00, Base.GenericCondition{Base.Threads.SpinLock}(Base.IntrusiveLinkedList{Task}(nothing, nothing), Base.Threads.SpinLock(0)), (0, 139838914076416, 139838913886120)), nothing)

In [39]:
t = @async fetch(x)

Task (done) @0x00007f2ec8356d60

In [40]:
istaskdone(t)

true

In [41]:
fetch(t)

20